In [1]:
import numpy as np

def getTile(tile):
    splitedTile = tile.split('\n')
    tileId = int(splitedTile[0].replace('Tile','').replace(':',''))
    return tileId, np.array([[j for j in i] for i in splitedTile[1:]])

In [2]:
fileInput = open('data.txt', mode='r')
dataInput = { getTile(tile)[0]:getTile(tile)[1] for tile in fileInput.read().split('\n\n') }
fileInput.close()

In [3]:
def getPossibleArrayFromArray(arr):
    rotatedArrays = [np.rot90(arr,k,(0,1)) for k in range(4)]
    flipArrays = [np.rot90(np.flip(arr,0),k,(0,1)) for k in range(4)]
    return rotatedArrays + flipArrays

In [4]:
import itertools as it

def getBordersOfTile(tile, borderSpec='all'):
    if(borderSpec == 'all'): return [tile[0],tile[-1],tile[:,0],tile[:,-1]]
    if(borderSpec == 'rb'): return [tile[-1],tile[:,-1]] # right & bottom

def testConnectedBorder(borders1, borders2):
    return any([ np.array_equal(b1,b2) for b1,b2 in it.product(borders1,borders2) ])

def areConnectedTiles(tile1,tile2, borderSpec='all'):
    bordersTile1 = getBordersOfTile(tile1, borderSpec)
    possibleBordersOfTile2 = [getBordersOfTile(t) for t in getPossibleArrayFromArray(tile2)]
    return any([testConnectedBorder(bordersTile1, bordersTile2) for bordersTile2 in possibleBordersOfTile2])

def countConnectedTiles(idTest,tiles,borderSpec='all'):
    s = 0
    for tileID, tileArray in tiles.items():
        if(tileID != idTest):
            if(areConnectedTiles(tiles[idTest],tileArray,borderSpec)): s+=1
    return s

In [5]:
from functools import reduce

def calculForPart1(tiles):
    cornerTiles = [ idTile for idTile in tiles.keys() if countConnectedTiles(idTile,tiles) == 2]
    return reduce(lambda a,b: a*b, cornerTiles), cornerTiles

In [6]:
from math import *

# First corner = top-left corner
def getFirstCorner(cornerTiles, tiles):
    cornerId = cornerTiles[0]
    while(countConnectedTiles(cornerId,tiles, 'rb') < 2):        
        tiles[cornerId] = np.rot90(tiles[cornerId], 1, (0,1))
    return tiles[cornerId], cornerId

# assume that currentTile is not in tiles
# currentTile is fixed (can't be rotated or flipped)
def findNextRightTile(currentTile, tiles):
    borderRight = currentTile[:,-1]
    for idTile, tile in tiles.items():
        possibleTiles = getPossibleArrayFromArray(tile)
        for t in possibleTiles:
            borderLeft = t[:,0]
            if(np.array_equal(borderRight,borderLeft)):
                return t, idTile

def findFirstTileOfLine(upTile, tiles):
    borderBottom = upTile[-1]
    for idTile, tile in tiles.items():
        possibleTiles = getPossibleArrayFromArray(tile)
        for t in possibleTiles:
            borderUp = t[0]
            if(np.array_equal(borderBottom,borderUp)):
                return t, idTile

'''
¤¤¤¤¤¤¤¤¤
¤¤ -->
|
v
we build puzzle line by line, and we start with anyone corner (the first for example)
that we rotate until it becomes the top-left corner of the puzzle
'''             
def constuctPuzzle(cornerTiles, tiles):
    squareLen = int(sqrt(len(tiles)))
    puzzle = []
    for i in range(squareLen):
        puzzleLine = []      
        corner, cornerID = getFirstCorner(cornerTiles, tiles) if(i==0) else findFirstTileOfLine(puzzle[-1][0], tiles)
        puzzleLine.append(corner); del tiles[cornerID]
        while(len(puzzleLine) < squareLen):
            nextTile, nextTileID = findNextRightTile(puzzleLine[-1],tiles)
            puzzleLine.append(nextTile); del tiles[nextTileID]
        puzzle.append(puzzleLine)
    return puzzle

In [7]:
from functools import reduce

def getFinalPuzzle(corners, tiles):
    puzzleTemp_1 = constuctPuzzle(corners, tiles)
    puzzleTemp_2 = [ [ tile[1:-1,1:-1] for tile in ligne] for ligne in puzzleTemp_1]
    puzzleTemp_3 = [reduce(lambda a,b: np.concatenate((a,b), axis=1), ligne) for ligne in puzzleTemp_2]
    return reduce(lambda a,b: np.concatenate((a,b), axis=0), puzzleTemp_3)

In [8]:
patternFile = open('pattern.txt', mode='r')
monsterPattern = [ [ char for char in line.replace('\n','')] for line in patternFile.readlines()]
def getPatternRule(pattern):
    L=[]
    for i in range(len(pattern)):
        for j in range(len(pattern[i])):
            if (pattern[i][j] == '#'): L.append((i,j))
    return L
nbLinePattern, nbColPattern = np.array(monsterPattern).shape
monsterPattern = getPatternRule(monsterPattern)
patternFile.close()

In [9]:
def isMatchingPattern(subImg):
    for i,j in monsterPattern:
        if(subImg[i,j] != '#'): return False
    return True

def imgHasMonsters(img):
    nbLine, nbCol = img.shape
    for i in range(nbLine-nbLinePattern):
        for j in range(nbCol-nbColPattern):
            if(isMatchingPattern(img[i:i+nbLinePattern,j:j+nbColPattern])): return True
    return False

def replaceMonsterByO(img):
    nbLine, nbCol = img.shape
    for i in range(nbLine-nbLinePattern):
        for j in range(nbCol-nbColPattern):
            if(isMatchingPattern(img[i:i+nbLinePattern,j:j+nbColPattern])):
                for x,y in monsterPattern: img[i+x,j+y] = 'O'
    return img

In [10]:
resPart1, corners = calculForPart1(dataInput)
correctPuzzle = [p for p in getPossibleArrayFromArray(getFinalPuzzle(corners,dataInput)) if imgHasMonsters(p)][0]

In [11]:
print('Part 1 :', resPart1)
print('Part 2 :', sum([l.count('#') for l in replaceMonsterByO(correctPuzzle).tolist()]))

Part 1 : 32287787075651
Part 2 : 1939
